# 3 Intro to PyLabRobot: Moving Labware on the OT2

## Learning Objectives:
**3.0.1** Moving Plates around the Deck

**3.0.2** Placing Lids

**3.0.3** Removing Lids

### Imports

In [1]:
import sys

In [2]:
# Make sure you are running Python version 3.10!
print(sys.version)

3.10.4 (main, Jun 29 2022, 12:14:53) [GCC 11.2.0]


In [3]:
import pylabrobot

In [13]:
# Importing necessary modules from the PyLabRobot package to handle liquids and␣visualize processes.
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer

# This import provides access to OTDeck, which represents the deck of an␣Opentrons robot.
from pylabrobot.resources.opentrons import OTDeck
# Loading and Plate Management

# Imports functions and classes to load resources and manage plate types␣specific to Opentrons robots.
from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

# Tracking and Contamination Prevention
# Enables tracking of tip usage, liquid volume, and helps prevent␣cross-contamination in experiments.
from pylabrobot.resources import set_tip_tracking, set_volume_tracking, set_cross_contamination_tracking
from pylabrobot.resources.plate import *

# Optional, use when interested in tracking the state of tips and volumes, generally keep this on
set_tip_tracking(True), set_volume_tracking(True)

# Optional, use when interested in protecting against accidental cross contamination
set_cross_contamination_tracking(True)

# External Libraries
# Importing standard libraries for additional functionality.
import opentrons # Provides access to Opentrons API for robot control.
import time # Allows for adding delays in the robot's operation for timing␣experiments.


##### Labware

In [5]:
from pylabrobot.resources import (
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    corning_96_wellplate_360ul_flat
)

### 3.0.1 Plate movement and Gripper Operations. 
#### If you ever get stuck, click the circley arrow at the top of the notebook to restart your progress!

First we will import `LiquidHandler`, a backend called `ChatterBoxBackend` that prints the text
output of our commands, a class `Visualizer` that provides a visualization of the robot deck as we
run commands, and a class `OTDeck()` that will represent the deck of an OpenTrons OT2, one of
the most widely used liquid handling robots. 

#### Set up `LiquidHandler` and `Visualizer`

In [6]:
lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

Setting up the robot.
Resource deck was assigned to the robot.
Resource trash_container was assigned to the robot.
Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


#### Setting up the Deck:
Let's begin this exercise by adding a 96-well plate and tube rack to the deck.

In [7]:
tube_rack = opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap("tube_rack")
plate = corning_96_wellplate_360ul_flat("plate")

lh.deck.assign_child_at_slot(tube_rack, 10)
lh.deck.assign_child_at_slot(plate, 3)

Resource tube_rack was assigned to the robot.
Resource plate was assigned to the robot.


In [ ]:
# Now, let's add some liquids to the first column of tubes
first_col_tubes = next(tube_rack.traverse(batch_size=4, direction='down'))
for i in range(len(first_col_tubes)):
    # [(liquid, volume)] is the proper syntax for adding a liquid to a labware. That's a list of tuples of the form (liquid_name, volume_in_uL)]
    first_col_tubes[i].tracker.set_liquids([(f"Dye_{i}", 2000)])

#### Introducing the Gripper!
Various liquid handling robots have a part called a `Gripper`. It is essentially an arm that you can program to move labware around the deck as your protocol progresses. The OpenTrons OT2 does not have one of these, but the OpenTrons Flex does. For this tutorial, we will be using a simulated `Gripper` class that implements the methods we need to demonstrate how you would use this feature in a real experiment.

* If you want to implement your own robot part, use this as inspiration and refer to this tutorial on [Python Classes](https://www.geeksforgeeks.org/python-classes-and-objects/).

In [25]:
class Gripper:
    # Constructor, this function instantiates our Gripper
    def __init__(self, lh):
        self.lh = lh

    # Move labware function, this function will take a labware and move it to dest_spot
    # Note, this will not work in a real PLR code, and is currently being implemented by the PLR team
    def move_labware(self, labware, dest_spot):
        self.lh.deck.unassign_child_resource(labware)
        self.lh.deck.assign_child_at_slot(labware, dest_spot)
        print(f"Moving {labware.name} to spot {dest_spot}")

    # Function to add a lid to a plate
    def add_lid(self, plate, lid, current_lid_spot=None):
        # If the lid has not been placed on the robot, keep current_lid_spot_none

        if current_lid_spot:
            lh.deck.unassign_child_resource(lid)
            plate.assign_child_resource(lid)
            print(f"Moving lid {lid.name} from spot {current_lid_spot} to plate {plate.name}")
        else:
            plate.assign_child_resource(lid)
            print(f"Adding lid {lid.name} to plate {plate.name}")

    # Function to remove a lid from a plate
    def remove_lid(self, plate, lid_dest_spot):
        assert plate.has_lid(), "Plate doesn't have a lid!"

        lid = plate.lid
        plate.unassign_child_resource(lid)
        lh.deck.assign_child_at_slot(lid, lid_dest_spot)

# Example declaration:
# Assuming you have a liquidHandler object lh
gripper = Gripper(lh)

In [26]:
gripper.move_labware(tube_rack, dest_spot = 2)

Resource tube_rack was unassigned from the robot.
Resource tube_rack was assigned to the robot.
Moving tube_rack to spot 2


#### Practice using the Gripper

Now that we know how to move labware, let's arrange our `tube rack` and `plate` in a way that makes sense for an experiment. Recall that we want to minimize contamination and fly-over spillage. Move the `tube rack` to spot 10, and the `plate` to spot 11.

In [ ]:
gripper.move_labware(tube_rack, dest_spot = ??)
gripper.move_labware($$$, dest_spot = ~~~)

In [52]:
lh.summary()


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: tube_rack   | 11: plate       | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: Empty       |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



### 3.0.2-3 Lids:

##### Lids are currently being worked on in PLR. We are going to hack our way around this issue for this tutorial, though. See [this pull request](https://github.com/PyLabRobot/pylabrobot/pull/161) for more reference.

In [24]:
# Save this code, we will use it to be our hacked Lid
plate_lid = Lid(name="Lid",
                size_x=127.0,
                size_y=86.0,
                size_z=0,           # measure the total z height
                nesting_z_height=1, # measure overlap between lid and plate
                model="Cos_96_FL_Lid")

##### Let's demonstrate how to move the lid with our `Gripper`

In [20]:
# Our plate_lid hasn't been added to the robot yet, so we don't have to specify a current_lid_spot
gripper.add_lid(plate=plate, lid=plate_lid)

Resource Lid was assigned to the robot.
Adding lid Lid to plate plate


In [21]:
# We can take off the lid with this function. Remember what you pick up, you must put down somewhere!
gripper.remove_lid(plate=plate, lid_dest_spot = 1)

Resource Lid was unassigned from the robot.
Resource Lid was assigned to the robot.


In [22]:
# Now that the lid lives on the deck somewhere, we need to specify the current_lid_spot.
gripper.add_lid(plate=plate, lid=plate_lid, current_lid_spot=2)

Resource Lid was unassigned from the robot.
Resource Lid was assigned to the robot.
Moving lid Lid from spot 2 to plate plate


In [23]:
lh.summary()


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: Empty       | 11: Empty       | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: Empty       |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: tube_rack   |  3: plate       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



### Exercises:

#### Exercise 1: Add another plate to the deck. Move the plate currently on the deck to spot 8, and the new plate to spot 5. We will refer to the plate on spot 5 as `Plate_1` and the plate on spot 8 as `Plate_2`.

#### Exercise 2: Move the `tube rack` to spot 7.

#### Exercise 3: Add lids to both plates. Make sure to declare new Lid objects, one for each plate.

#### Exercise 4: In one cell, write a liquid handling protocol that accomplishes the following steps:
1. Remove lid from `Plate_1`.
2. Add 200 uL of Dye to `Column 1` of `Plate_1`.
3. Place a lid on `Plate_1`.
4. Swap locations of `Plate_1` and `Plate_2`.